<a href="https://colab.research.google.com/github/SFIComplexityExplorer/Mesa-ABM-Tutorial/blob/main/Session_15_Traders_Trade_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


try:
  import mesa
except:
  !pip install mesa --quiet
import mesa
import numpy as np
import math
import matplotlib.pyplot as plt

%matplotlib inline

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.1 MB/s eta 0:00:00


# Helper Functions

In [ ]:
def get_distance(pos_1, pos_2):
  '''
  Calculate the Euclidean distance between two positions

  used in trade.move()
  '''

  x1, y1 = pos_1
  x2, y2 = pos_2
  dx = x1 - x2
  dy = y1 - y2
  return math.sqrt(dx**2 + dy**2)

# Resource Classes

In [ ]:
class Sugar(mesa.Agent):
  """
  Sugar:
  - contains an amount of sugar
  - grows 1 amount of sugar at each turn
  """

  def __init__(self, unique_id, model, pos, max_sugar):
    super().__init__(unique_id, model)
    self.pos = pos
    self.amount = max_sugar
    self.max_sugar = max_sugar


  def step(self):
    '''
    Sugar growth function, adds one unit of sugar each step until
    max amount
    '''
    self.amount = min([self.max_sugar, self.amount+1])

In [ ]:
class Spice(mesa.Agent):
  """
  Spice:
  - contains an amount of spice
  - grows 1 amount of spice at each turn
  """

  def __init__(self, unique_id, model, pos, max_spice):
    super().__init__(unique_id, model)
    self.pos = pos
    self.amount = max_spice
    self.max_spice = max_spice

  def step(self):
    '''
    Spice growth function, adds one unit of spice each step until
    max amout
    '''
    self.amount = min([self.max_spice, self.amount+1])

# Trader Agent

In [ ]:
class Trader(mesa.Agent):
  """
  Trader:
  - has a metabolism of sugar and spice
  - harvest and trade sugar and spice to survive
  """


  def __init__(self, unique_id, model, pos, moore=False, sugar=0,
               spice=0, metabolism_sugar=0, metabolism_spice=0,
               vision=0):
    super().__init__(unique_id, model)
    self.pos = pos
    self.moore = moore
    self.sugar = sugar
    self.spice = spice
    self.metabolism_sugar = metabolism_sugar
    self.metabolism_spice = metabolism_spice
    self.vision = vision


  def get_sugar(self, pos):
    '''
    used in self.get_sugar_amount()
    '''

    this_cell = self.model.grid.get_cell_list_contents(pos)
    for agent in this_cell:
      if type(agent) is Sugar:
        return agent
    return None


  def get_sugar_amount(self,pos):
    '''
    used in self.move() as part of self.calculate_welfare()
    '''

    sugar_patch = self.get_sugar(pos)
    if sugar_patch:
      return sugar_patch.amount
    return 0

  def get_spice(self, pos):
    '''
    used in self.get_spice_amount()
    '''

    this_cell = self.model.grid.get_cell_list_contents(pos)
    for agent in this_cell:
      if type(agent) is Spice:
        return agent
    return None

  def get_spice_amount(self, pos):
    '''
    used in self.move() as part of self.calculate_welfare()
    '''

    spice_patch = self.get_spice(pos)
    if spice_patch:
      return spice_patch.amount
    return 0

  def get_trader(self, pos):
      '''
      helper function used in self.trade_with_neighbors()
      '''

      this_cell = self.model.grid.get_cell_list_contents(pos)

      for agent in this_cell:
        if isinstance(agent, Trader):
          return agent


  def is_occupied_by_other(self,pos):
    '''
    helper function part 1 of self.move()
    '''

    if pos == self.pos:
      # agent's position is considered unoccupied as agent can stay there
      return False
    # get contents of each cell in neighborhood
    this_cell = self.model.grid.get_cell_list_contents(pos)
    for a in this_cell:
      # see if occupied by another agent
      if isinstance(a, Trader):
        return True
    return False

  def calculate_welfare(self, sugar, spice):
    '''
    helper function part 2 self.move()
    '''

    # calculate total resources
    m_total = self.metabolism_sugar + self.metabolism_spice
    # Cobb-Douglas functional form; starting on p. 97 on Growing Artificial Societies
    return sugar**(self.metabolism_sugar/m_total) * spice**(
        self.metabolism_spice/m_total)


  def is_starved(self):
    '''
    Helper function for self.maybe_die()
    '''

    return (self.sugar <= 0) or (self.spice <= 0)

  def calculate_MRS(self, sugar, spice):
    '''
    Helper function for self.trade()

    Determines what trader agent is needs and can give up
    '''

    return (spice/self.metabolism_spice) / (sugar/self.metabolism_sugar)


  def trade(self, other):
    '''
    helper function used in trade_with_neighbors()

    other is a trader agent object
    '''

    # sanity check to verify code is working as expected
    assert self.sugar > 0
    assert self.spice > 0
    assert other.sugar > 0
    assert other.spice > 0

    # calculate marginal rate of subsitution in Growing Artificial Socieites p. 101
    mrs_self = self.calculate_MRS(self.sugar, self.spice)
    mrs_other = other.calculate_MRS(other.sugar, other.spice)

    # calculate each agents welfare in Growing Artifcial Socieites p. 97
    welfare_self = self.calculate_welfare(self.sugar, self.spice)
    welfare_other = other.calculate_welfare(other.sugar, other.spice)

    if math.isclose(mrs_self, mrs_other, rel_tol=1e-02):
      return

    price = math.sqrt(mrs_self*mrs_other)
    print(price)

  ######################################################################
  #                                                                    #
  #                      MAIN TRADE FUNCTIONS                          #
  #                                                                    #
  ######################################################################


  def move(self):
    '''
    Function for trader agent to identify optimal move for each step in 4 parts
    1 - identify all possible moves
    2 - determine which move maximizes welfare
    3 - find closest best option
    4 - move
    '''

    # 1. identify all possible moves

    neighbors = [i
                 for i in self.model.grid.get_neighborhood(
                   self.pos, self.moore, True, self.vision
                 ) if not self.is_occupied_by_other(i)]

    # 2. determine which move maximizes welfare

    welfares = [
        self.calculate_welfare(
            self.sugar + self.get_sugar_amount(pos),
            self.spice + self.get_spice_amount(pos))
        for pos in neighbors
    ]

    # 3. Find closest best option

    # find the highest welfare in welfares
    max_welfare = max(welfares)
    # get the index of max welfare cells
    candidate_indices = [i for i in range(len(welfares))
                        if math.isclose(welfares[i], max_welfare,
                                        rel_tol=1e-02)]

    # convert index to positions of those cells
    candidates = [neighbors[i] for i in candidate_indices]

    min_dist = min(get_distance(self.pos, pos) for pos in candidates)

    final_candidates = [ pos for pos in candidates
                       if math.isclose(get_distance(self.pos, pos), min_dist, rel_tol=1e-02
                                       )]
    self.random.shuffle(final_candidates)

    # 4. Move Agent
    self.model.grid.move_agent(self, final_candidates[0])

  def eat(self):
    # get sugar
    sugar_patch = self.get_sugar(self.pos)

    if sugar_patch:
        self.sugar += sugar_patch.amount
        sugar_patch.amount = 0
    self.sugar -= self.metabolism_sugar

    # get_spice
    spice_patch = self.get_spice(self.pos)

    if spice_patch:
        self.spice += spice_patch.amount
        spice_patch.amount = 0
    self.spice -= self.metabolism_spice

  def maybe_die(self):
    '''
    Function to remove Traders who have consumed all their sugar or spice
    '''

    if self.is_starved():
      self.model.grid.remove_agent(self)
      self.model.schedule.remove(self)

  def trade_with_neighbors(self):
    '''
    Function for trader agents to decide who to trade with in three parts

    1- identify neighbors who can trade
    2- trade (2 sessions)
    3- collect data
    '''

    neighbor_agents = [self.get_trader(pos) for pos in self.model.grid.get_neighborhood(
        self.pos, self.moore, False, self.vision) if self.is_occupied_by_other(pos)
    ]

    if len(neighbor_agents) == 0:
      # trade returns price and trade network (who agent traded with)
      return [], []


    # iterate through traders in neighboring cells and trade
    for a in neighbor_agents:
      if a:
        self.trade(a)


# Model Class

In [ ]:
class SugarscapeG1mt(mesa.Model):
  """
  Manager class to run Sugarscape with Traders
  """


  def __init__(self, width=50,height=50, initial_population=200,
               endowment_min=25, endowment_max=50, metabolism_min=1,
               metabolism_max=5, vision_min=1, vision_max=5):

    #Initiate width and heigh of sugarscape
    self.width = width
    self.height = height
    #Initiate population attributes
    self.initial_population = initial_population
    self.endowment_min = endowment_min
    self.endowment_max = endowment_max
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max

    #initiate activation schedule
    self.schedule = mesa.time.RandomActivationByType(self)
    #initiate mesa grid class
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

    #read in landscape file from supplmentary material
    sugar_distribution =np.genfromtxt("/content/drive/MyDrive/sugar-map.txt")
    spice_distribution = np.flip(sugar_distribution, 1)

    agent_id = 0
    for _,(x,y) in self.grid.coord_iter():
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0:
        sugar = Sugar(agent_id, self, (x,y), max_sugar)
        self.schedule.add(sugar)
        self.grid.place_agent(sugar, (x,y))
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0:
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.schedule.add(spice)
        self.grid.place_agent(spice, (x,y))
        agent_id += 1

    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width)
      y = self.random.randrange(self.height)
      #see Growing Artificial Societies p. 108 for initialization
      #give agents initial endowment
      sugar = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
      spice = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
      #give agents initial metabolism
      metabolism_sugar = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
      metabolism_spice = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
      #give agents vision
      vision = int(self.random.uniform(self.vision_min, self.vision_max+1))
      #create Trader object
      trader = Trader(agent_id,
                      self,
                      (x,y),
                      moore = False,
                      sugar = sugar,
                      spice = spice,
                      metabolism_sugar = metabolism_sugar,
                      metabolism_spice = metabolism_spice,
                      vision = vision)
      #place agent
      self.grid.place_agent(trader, (x,y))
      self.schedule.add(trader)
      agent_id += 1

  def randomize_traders(self):
    '''
    helper function for self.step()

    puts traders in randomized list for step function
    '''

    trader_shuffle = list(self.schedule.agents_by_type[Trader].values())
    self.random.shuffle(trader_shuffle)

    return trader_shuffle


  def step(self):
    '''
    Unique step function that does staged activation of sugar and spice
    and then randomly activates traders
    '''
    # step Sugar agents
    for sugar in self.schedule.agents_by_type[Sugar].values():
      sugar.step()

    # step Spice agents
    for spice in self.schedule.agents_by_type[Spice].values():
      spice.step()

    # step trader agents
    # to account for agent death and removal we need a seperate data strcuture to
    # iterate
    trader_shuffle = self.randomize_traders()

    for agent in trader_shuffle:
      agent.move()
      agent.eat()
      agent.maybe_die()

    trader_shuffle = self.randomize_traders()

    for agent in trader_shuffle:
      agent.trade_with_neighbors()

    self.schedule.steps += 1 #important for data collector to track number of steps

  def run_model(self, step_count=1000):

    for i in range(step_count):
      self.step()

# Run Sugarscape

In [ ]:
model = SugarscapeG1mt()
model.run_model(step_count=1)


3.442331661289181
1.531855696857129
1.1357816691600546
0.8017837257372731
0.9309493362512627
0.7303459265401521
1.1353688827865596
2.518487646781408
0.7485114296183861
1.0327955589886444
0.6839855680567695
1.4907119849998596
2.642193697809283
3.586095690932793
1.7568209223157663
0.9296222517045284
3.442331661289181
0.2517150185491601
0.6332313927204475
0.9980750703999405
1.5879470976674945
1.7315460572135377
1.7783118855794566
0.5568460463897045
0.6784669927988098
2.139692227467032
3.404271044608362
1.7315460572135377
1.8136906252750293
1.1091021264436023
2.8042520905286037
1.3814367980063893
1.1091021264436023
1.4137281621314504
1.0568922729378547
1.6605279103876789
0.806225774829855
1.2260639292551958
2.5155764746872635
0.9756654533819864
1.05777177069589
0.8902476018169285
0.6477502756312957
1.260797521258029
0.4481341047322507
1.3064825110670781
0.7303459265401521
0.9309493362512627
4.621695548948082
1.6339011369947531
2.125118592516207
1.0516609640992376
0.5429366627703638
0.67846